First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import torch
import torchvision
from torch import nn

CATEGORIES = ['hand', 'TennisBall']

device = torch.device('cuda')
model_classification = torchvision.models.resnet18(pretrained=False)
model_classification.fc = torch.nn.Linear(512, len(CATEGORIES))
#model_classification.fc= nn.Sequential(nn.Linear(512, 256), nn.ReLU(), nn.Dropout(0.2), nn.Linear(256, len(CATEGORIES)),nn.LogSoftmax(dim=1))

model_classification = model_classification.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [2]:
model_classification.load_state_dict(torch.load('classification_nonormalize_backgorund_2cat_20epochs_HandTennis.pth'))

<All keys matched successfully>

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [3]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_classification_trt = torch2trt(model_classification, [data], fp16_mode=True)

Save the optimized model using the cell below

In [4]:
torch.save(model_classification_trt.state_dict(), 'trt_classification_nonormalize_backgorund_2cat_20epochs_HandTennis.pth')

Load the optimized model by executing the cell below

In [5]:
import torch
from torch2trt import TRTModule

model_classification_trt = TRTModule()
model_classification_trt.load_state_dict(torch.load('trt_classification_nonormalize_backgorund_2cat_20epochs_HandTennis.pth'))

<All keys matched successfully>

Create the camera class.

In [6]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)

In [7]:
#PRED WITHOUT WHEELS MOVING 
from utils import preprocess
import numpy as np
import time
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)

STEERING_GAIN = 0.95
STEERING_BIAS = -0.30

#car.throttle = 0.0

while True:
    
    image = camera.read()
    image = preprocess(image).half()
    #output = model_classification(image).detach().cpu().numpy().flatten()
    prediction_ = model_classification(image)
    prediction_prob=torch.nn.functional.softmax(prediction_,dim=1)
    prediction_prob=prediction_prob*100
    
    print("prob",prediction_prob.detach().cpu().numpy().flatten())
    category_number = CATEGORIES[prediction_prob.argmax()]
    print(category_number)
    print("\n")
    
    time.sleep(0.5)

prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0. 100.]
TennisBall


prob [  0.

KeyboardInterrupt: 